学籍番号：20M30574
名前：朱嘉基

In [1]:
import pandas as pd

# CSVファイルの読み込み
def load_data():
  return pd.read_csv("train.csv"), pd.read_csv("test.csv")

train_data, test_data = load_data()

In [2]:
x_train = train_data.loc[:, "Alcohol":]
x_test = test_data.loc[:, "Alcohol":]
y_train = train_data["Class"]
y_test = test_data["Class"]

In [3]:
import numpy as np
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [4]:
steps = [
    ('sc', StandardScaler()),
    ('pca', PCA()),
    ('hist', HistGradientBoostingClassifier())
]
pipeline = Pipeline(steps = steps)

In [5]:
parameters = {
    "pca__n_components" : [i for i in range(2, 13)],
    "hist__learning_rate" : [0.001 * i for i in range(1, 21)]
}

In [6]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
grid = GridSearchCV(pipeline, param_grid = parameters, cv = kf, verbose = 2, n_jobs = 2, scoring = "accuracy")
grid.fit(x_train, y_train)
result = grid.cv_results_

Fitting 5 folds for each of 220 candidates, totalling 1100 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    6.6s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:   26.1s
[Parallel(n_jobs=2)]: Done 361 tasks      | elapsed:   58.4s
[Parallel(n_jobs=2)]: Done 644 tasks      | elapsed:  1.7min
[Parallel(n_jobs=2)]: Done 1009 tasks      | elapsed:  2.8min
[Parallel(n_jobs=2)]: Done 1100 out of 1100 | elapsed:  3.1min finished


In [7]:
rank = np.vstack((result["mean_test_score"], result["params"]))
rank = rank[:, np.argsort(rank[0])[::-1]]
np.set_printoptions(threshold=np.inf)
print(np.transpose(rank))

[[0.9507389162561577
  {'hist__learning_rate': 0.015, 'pca__n_components': 2}]
 [0.9507389162561577
  {'hist__learning_rate': 0.016, 'pca__n_components': 2}]
 [0.9507389162561577
  {'hist__learning_rate': 0.014, 'pca__n_components': 2}]
 [0.9507389162561577
  {'hist__learning_rate': 0.013000000000000001, 'pca__n_components': 2}]
 [0.9504926108374384
  {'hist__learning_rate': 0.015, 'pca__n_components': 4}]
 [0.9504926108374384
  {'hist__learning_rate': 0.011, 'pca__n_components': 4}]
 [0.9504926108374384
  {'hist__learning_rate': 0.014, 'pca__n_components': 4}]
 [0.9504926108374384
  {'hist__learning_rate': 0.014, 'pca__n_components': 3}]
 [0.9504926108374384
  {'hist__learning_rate': 0.013000000000000001, 'pca__n_components': 4}]
 [0.9504926108374384
  {'hist__learning_rate': 0.013000000000000001, 'pca__n_components': 3}]
 [0.9504926108374384
  {'hist__learning_rate': 0.012, 'pca__n_components': 4}]
 [0.9504926108374384
  {'hist__learning_rate': 0.015, 'pca__n_components': 3}]
 [0.943

In [8]:
y_pred = grid.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

0.9722222222222222
